In [1]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [2]:
#install a Python-accessible MondoDB package

#!conda install pymongo

In [3]:
import pymongo

from pymongo import MongoClient
#mongo read write directory 
#mongo--dbpath /usr/local/mongodb-data


In [4]:
client = MongoClient('localhost', 27017)

In [5]:
# following tutorial at:
# https://realpython.com/introduction-to-mongodb-and-python/#using-mongodb-with-python-and-pymongo

db = client['rptutorials']

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'rptutorials')

In [6]:
tutorial1 = {
     "title": "Working With JSON Data in Python",
     "author": "Lucas",
     "contributors": [
         "Aldren",
         "Dan",
         "Joanna"
     ],
     "url": "https://realpython.com/python-json/"
}

In [7]:
tutorial = db.tutorial
tutorial

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'rptutorials'), 'tutorial')

In [8]:
result = tutorial.insert_one(tutorial1)
result

In [9]:
tutorial2 = {
     "title": "Python's Requests Library (Guide)",
     "author": "Alex",
     "contributors": [
         "Aldren",
         "Brad",
         "Joanna"
     ],
     "url": "https://realpython.com/python-requests/"
}

tutorial3 = {
     "title": "Object-Oriented Programming (OOP) in Python 3",
     "author": "David",
     "contributors": [
         "Aldren",
         "Joanna",
         "Jacob"
     ],
     "url": "https://realpython.com/python3-object-oriented-programming/"
}

In [10]:
new_result = tutorial.insert_many([tutorial2, tutorial3])

print(f"Multiple tutorials: {new_result.inserted_ids}")

Multiple tutorials: [ObjectId('64e8c38c12d81a65e0c2f9a4'), ObjectId('64e8c38c12d81a65e0c2f9a5')]


In [11]:
import pprint

for doc in tutorial.find():
    pprint.pprint(doc)

{'_id': ObjectId('64dcd099076a5f8a9e7379d9'),
 'author': 'Lucas',
 'contributors': ['Aldren', 'Dan', 'Joanna'],
 'title': 'Working With JSON Data in Python',
 'url': 'https://realpython.com/python-json/'}
{'_id': ObjectId('64dcd142076a5f8a9e7379da'),
 'author': 'Alex',
 'contributors': ['Aldren', 'Brad', 'Joanna'],
 'title': "Python's Requests Library (Guide)",
 'url': 'https://realpython.com/python-requests/'}
{'_id': ObjectId('64dcd142076a5f8a9e7379db'),
 'author': 'David',
 'contributors': ['Aldren', 'Joanna', 'Jacob'],
 'title': 'Object-Oriented Programming (OOP) in Python 3',
 'url': 'https://realpython.com/python3-object-oriented-programming/'}
{'_id': ObjectId('64e4b984ebe8017a0ff48cfb'),
 'author': 'Lucas',
 'contributors': ['Aldren', 'Dan', 'Joanna'],
 'title': 'Working With JSON Data in Python',
 'url': 'https://realpython.com/python-json/'}
{'_id': ObjectId('64e4b984ebe8017a0ff48cfc'),
 'author': 'Alex',
 'contributors': ['Aldren', 'Brad', 'Joanna'],
 'title': "Python's Requ

In [12]:
jon_tutorial = tutorial.find_one({"author": "Alex"})
pprint.pprint(jon_tutorial)

{'_id': ObjectId('64dcd142076a5f8a9e7379da'),
 'author': 'Alex',
 'contributors': ['Aldren', 'Brad', 'Joanna'],
 'title': "Python's Requests Library (Guide)",
 'url': 'https://realpython.com/python-requests/'}


In [13]:
# example of inserting data to MondoDB from csv file

import pandas as pd
import json

def mongoimport(csv_path, db_name, coll_name, db_url='localhost', db_port=27017):
    """ Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = pd.read_csv(csv_path)
    payload = json.loads(data.to_json(orient='records'))
    coll.remove()
    coll.insert(payload)
    return coll.count()

db = client['zillo']

z_data = db.z_data
z_data

mongoimport("Zillow_Mortgages_county_2016-2017.csv", "zillo", "z_data")

/var/folders/d9/dyxm3y8j7bgd2x_4hx7tt0200000gn/T/ipykernel_37462/652687602.py:15: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  coll.remove()
/var/folders/d9/dyxm3y8j7bgd2x_4hx7tt0200000gn/T/ipykernel_37462/652687602.py:16: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  coll.insert(payload)
/var/folders/d9/dyxm3y8j7bgd2x_4hx7tt0200000gn/T/ipykernel_37462/652687602.py:17: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  return coll.count()


64575

In [14]:
#result = z_data.find_one({"DaysOnZillow_AllHomes": 130.5})
#result = z_data.find_one({"InventoryRaw_AllHomes": 846})
results = z_data.find({"InventoryRaw_AllHomes": {"$lt": 30}})

for result in results:
    print(result)

#pprint.pprint(result)

{'_id': ObjectId('64e8c38f12d81a65e0c2f9bb'), 'Date': '2016-01-31', 'RegionName': 1041, 'DaysOnZillow_AllHomes': None, 'InventorySeasonallyAdjusted_AllHomes': 29.0, 'InventoryRaw_AllHomes': 28.0, 'MedianListingPricePerSqft_1Bedroom': None, 'MedianListingPricePerSqft_2Bedroom': None, 'MedianListingPricePerSqft_3Bedroom': None, 'MedianListingPricePerSqft_4Bedroom': None, 'MedianListingPricePerSqft_5BedroomOrMore': None, 'MedianListingPricePerSqft_AllHomes': None, 'MedianListingPricePerSqft_CondoCoop': None, 'MedianListingPricePerSqft_DuplexTriplex': None, 'MedianListingPricePerSqft_SingleFamilyResidence': None, 'MedianListingPrice_1Bedroom': None, 'MedianListingPrice_2Bedroom': None, 'MedianListingPrice_3Bedroom': None, 'MedianListingPrice_4Bedroom': None, 'MedianListingPrice_5BedroomOrMore': None, 'MedianListingPrice_AllHomes': None, 'MedianListingPrice_CondoCoop': None, 'MedianListingPrice_DuplexTriplex': None, 'MedianListingPrice_SingleFamilyResidence': None, 'MedianPctOfPriceReductio

In [15]:
record_count = z_data.count_documents({"InventoryRaw_AllHomes": {"$lt": 30}})
print(f"Number of records matching the query: {record_count}")




Number of records matching the query: 804


In [16]:
import statsmodels.api as sm
import pandas as pd


# Retrieve data and create a pandas DataFrame
data = list(z_data.find({"InventorySeasonallyAdjusted_AllHomes": {"$lt": 30}}))

df = pd.DataFrame(data)


# Handle missing values by imputing with the mean
  # Drops rows with any missing values
df.fillna(df.mean(), inplace=True)



# Perform linear regression
X = df[["MedianListingPricePerSqft_AllHomes", "InventorySeasonallyAdjusted_AllHomes"]]
#X = df[["InventoryRaw_AllHomes"]]
X = sm.add_constant(X)  # Add a constant term for intercept
y = df["DaysOnZillow_AllHomes"]

model = sm.OLS(y, X).fit()

# Print regression results
print(model.summary())






                              OLS Regression Results                             
Dep. Variable:     DaysOnZillow_AllHomes   R-squared:                         nan
Model:                               OLS   Adj. R-squared:                    nan
Method:                    Least Squares   F-statistic:                       nan
Date:                   Fri, 25 Aug 2023   Prob (F-statistic):                nan
Time:                           11:06:59   Log-Likelihood:                    nan
No. Observations:                    790   AIC:                               nan
Df Residuals:                        787   BIC:                               nan
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

/var/folders/d9/dyxm3y8j7bgd2x_4hx7tt0200000gn/T/ipykernel_37462/1249034365.py:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [17]:
client.close()